In [1]:
#print('get stamps')
#!wget https://lasair.roe.ac.uk/lasair/static/ztf/stamps/fits/stamps770.tar
#print('fetched')
#!mkdir fits
#!cd fits; tar xf stamps770.tar
#!mkdir jpg
#print('unpacked')

In [2]:
import os, sys
from astropy.io import fits
from PIL import Image
import numpy
import os.path
import time

In [3]:
t_read = t_write = 0.0

In [4]:
def open_fits(filename):
    global t_read
    t = time.time()
    data = fits.getdata(filename)
    t_read += time.time() - t

    img = Image.fromarray(data)
    y=numpy.asarray(img.getdata(),dtype=numpy.float64).reshape((img.size[1],img.size[0]))

    y = numpy.nan_to_num(y)
    numpy.seterr(invalid='ignore')
#    print numpy.isnan(y)
    y = numpy.abs(y)
#    y = numpy.sqrt(y)
    med = numpy.median(y)
    mymin = 0
    mymax = 3*med
#    print '%.1f %s' % (med, filename)
    y = (y-mymin)*255/(mymax-mymin)
    y = numpy.maximum(y, 0)
    y = numpy.minimum(y, 255)
    y=numpy.asarray(y,dtype=numpy.uint8)
    return Image.fromarray(y,mode='L')

In [ ]:
def convert_fits(dirfits, dirjpg, name):
    global t_write
    t = name.split('_')
    file_ref     = '%s/%s_ref.fits.gz' % (dirfits, t[0])
    file_sci     = '%s/%s_%s_targ_sci.fits.gz' % (dirfits, t[0], t[1])
    file_scimref = '%s/%s_%s_targ_scimref.fits.gz' % (dirfits, t[0], t[1])
    file_refmsci = '%s/%s_%s_targ_refmsci.fits.gz' % (dirfits, t[0], t[1])
    file_jpg     = '%s/%s.jpg' % (dirjpg, t[0])
    if os.path.isfile(file_jpg):
        return 0
    
    border = 5
    size = 63
    new = Image.new('L', (4*border+3*size,2*border+size))
    
    sci = open_fits(file_sci)
    ref = open_fits(file_ref)
    try:    diff = open_fits(file_scimref)
    except: diff = open_fits(file_refmsci)

    new.paste(sci, (border, border))
    new.paste(ref, (size+2*border, border))
    new.paste(diff, (2*size+3*border,border))
    
    t = time.time()
    new.save(file_jpg, 'jpeg')
    t_write += time.time() - t
    return 1

In [ ]:
dirfits = '770/fits'
dirjpg  = '770/jpg'

day_total = 0
run_total = 0
t = time.time()

if not os.path.exists(dirjpg):
    print('making jpg directory')
    os.system('mkdir %s' % dirjpg)

if os.path.exists(dirfits):
    for file in os.listdir(dirfits):
        if file.endswith('_targ_sci.fits.gz'):
            day_total += 1
            run_total += convert_fits(dirfits, dirjpg, file)
print('------------  JPG STAMPS ----------')
print('Tried %d stamps, Made %d jpegs' % (day_total, run_total))
print('Time %.1f, read %.1f, write %.1f ' % ((time.time() - t), t_read, t_write))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: RuntimeWarning: divide by zero encountered in true_divide
